# Importing stuff

In [1]:
from calimba.analysis.utilities import add_folder_to_path

In [2]:
add_folder_to_path('/home/anleg84/GitHub/Calimba/')

In [3]:
from timeseries import *
from utilities import *
from brainstates import *
from visualization import *
from data import *
from atlas import *


from pathlib import Path
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import mat73

plt.rcParams['font.size'] = 20

/home/anleg84/anaconda3/envs/calimba/lib/python3.9/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)


In [4]:
darkblue = "#253C78"
emerald = "#68D89B"
red = "#F63E02"

# Setting paths

In [5]:
valeria_path = '/home/anleg84/Documents/Valeria/Data/'
results_path = '/home/anleg84/Documents/Code/NetworksPaper2023/Results/'
figures_path = '/home/anleg84/Documents/Code/NetworksPaper2023/Figures/'
atlas_path = '/home/anleg84/Documents/Atlas/Mapzebrain/'

In [6]:
atlas = MapzebrainAtlas(atlas_path)

In [7]:
directories = identify_folders(valeria_path, ['dpf'])
directories

['/home/anleg84/Documents/Valeria/Data/2021-09-16-Wholebrain-8dpf/',
 '/home/anleg84/Documents/Valeria/Data/2021-09-21-Wholebrain-6dpf/',
 '/home/anleg84/Documents/Valeria/Data/2021-09-22-Wholebrain-7dpf/',
 '/home/anleg84/Documents/Valeria/Data/2022-01-18-Wholebrain-6dpf/',
 '/home/anleg84/Documents/Valeria/Data/2022-02-01-Wholebrain-6dpf/',
 '/home/anleg84/Documents/Valeria/Data/2022-02-02-Wholebrain-7dpf/',
 '/home/anleg84/Documents/Valeria/Data/2022-02-22-Wholebrain-6dpf/',
 '/home/anleg84/Documents/Valeria/Data/2022-02-23-Wholebrain-7dpf/',
 '/home/anleg84/Documents/Valeria/Data/2022-02-24-Wholebrain-8dpf/',
 '/home/anleg84/Documents/Valeria/Data/2022-04-12-Wholebrain-6dpf/',
 '/home/anleg84/Documents/Valeria/Data/2022-04-13-Wholebrain-7dpf/',
 '/home/anleg84/Documents/Valeria/Data/2022-07-26-Wholebrain-6dpf/',
 '/home/anleg84/Documents/Valeria/Data/2022-07-27-Wholebrain-7dpf/',
 '/home/anleg84/Documents/Valeria/Data/2022-09-20-Wholebrain-6dpf/',
 '/home/anleg84/Documents/Valeria/

# Setting paths to Ahrens data

In [8]:
from scipy.io import loadmat

In [9]:
def get_stim_sequence(data):
    list_of_arrays = list(data['sequence'][0])
    sequence = []
    for element in list_of_arrays:
        sequence.append(element[0])
    return np.array(sequence)

def get_sequence_vector(data, stim):
    stim_sequence = get_stim_sequence(data)
    sequence_vector = np.zeros((data['timeSeries'].shape[1], ))
    for i, s in enumerate(stim_sequence):
        if s == stim:
            sequence_vector[data['intervals'][i][0]:data['intervals'][i][1]] = 1
    return sequence_vector

In [10]:
path = '/home/anleg84/Documents/Valeria/Data/Lightsheet-Ahrens/'
files = identify_files(path, ['.mat'])

In [11]:
path_stack = '/home/anleg84/Documents/Data/Registration/Atlases/Registration-Atlases2/ZBrain_elavl3.nrrd'
stack, _ = load_stack(path_stack)

# Transforming centroids into Mapzebrain atlas

The registration is imperfect at the dorso-caudal end of the hindbrain, but this should suffice for a first comparison. Registration will have to be re-done using multi-channel alignment.

#### Single fish inspection

In [23]:
stack.shape

(138, 621, 1406)

In [121]:
file = files[8]

In [122]:
try:
    data = loadmat(path + file)
except:
    data = mat73.loadmat(path + file)

In [123]:
centroids = np.copy(data['XYZ'])

In [124]:
centroids[:, 0] = 1406 - centroids[:, 0] # Flipping x axis
#centroids[:, 1] = 621 - centroids[:, 1] # Flipping y axis
centroids[:, 2] = 138 - centroids[:, 2] # Flipping z axis

In [127]:
%matplotlib inline

In [126]:
plt.figure(figsize=(20, 10))
plt.imshow(np.mean(stack, axis=0), cmap='gray')
plt.scatter(centroids[:, 0], centroids[:, 1], alpha=0.1, color='red')
#plt.xlim([200, 600])
#plt.ylim([100, 500])

Batch transformation

In [128]:
from registration.registration import *

In [129]:
ANTs.path = '/home/anleg84/Documents/ANTs/bin/'
path_to_transforms = '/home/anleg84/Documents/Data/Registration/Atlases/Registration-Atlases2/'

In [130]:
for i, file in enumerate(files):
    
    print(file)
    
    subject = file.split('.')[0]
    try:
        data = loadmat(path + file)
    except:
        data = mat73.loadmat(path + file)
        
    centroids = data['XYZ']
    
    centroids[:, 0] = 1406 - centroids[:, 0] # Flipping x axis
    centroids[:, 2] = 138 - centroids[:, 2] # Flipping z axis
    
    centroids[:, 0] = 0.7980005 * centroids[:, 0]
    centroids[:, 1] = 0.7980005 * centroids[:, 1]
    centroids[:, 2] = 2 * centroids[:, 2]
    
    write_coordinates_to_csv(path + 'coordinates_zbrain_{}.csv'.format(subject), centroids)
    
    ants = ANTs(path)
    ants.applyTransformsToPoints_inverse('coordinates_zbrain_{}.csv'.format(subject),
                                 'coordinates_mapzebrain_{}.csv'.format(subject),
                                 path_to_transforms)

subject10.mat
subject11.mat
subject12.mat
subject13.mat
subject14.mat
subject15.mat
subject16.mat
subject17.mat
subject18.mat
subject8.mat
subject9.mat


# Mapping centroids into Mapzebrain atlas

In [131]:
atlas = MapzebrainAtlas(atlas_path)

In [133]:
for i, file in enumerate(files):
                    
    subject = file.split('.')[0]

    print('Loading centroids...')
    coordinates = load_coordinates_csv(path + 'coordinates_mapzebrain_{}.csv'.format(subject))
    centroids = rescale_centroids(coordinates, [0.9939617, 0.9940714, 1.9944444])
    centroids[:, 2] *= 359/180 # Atlas was scaled 0.5 along y-axis
    centroids = centroids.astype('int')

    print('Mapping neurons in brain regions...')
    region_labels = atlas.mapCentroids(centroids, orientation='horizontal')

    print('Saving data...')
    np.save(path + 'centroids_mapzebrain_{}.npy'.format(subject), centroids)
    np.save(path + 'region_labels_mapzebrain_{}.npy'.format(subject), region_labels)

Loading centroids...
Mapping neurons in brain regions...
100%|███████████████████████████████████████████| 70/70 [00:28<00:00,  2.43it/s]
Saving data...
Loading centroids...
Mapping neurons in brain regions...
100%|███████████████████████████████████████████| 70/70 [00:27<00:00,  2.57it/s]
Saving data...
Loading centroids...
Mapping neurons in brain regions...
100%|███████████████████████████████████████████| 70/70 [00:27<00:00,  2.56it/s]
Saving data...
Loading centroids...
Mapping neurons in brain regions...
100%|███████████████████████████████████████████| 70/70 [00:27<00:00,  2.55it/s]
Saving data...
Loading centroids...
Mapping neurons in brain regions...
100%|███████████████████████████████████████████| 70/70 [00:27<00:00,  2.56it/s]
Saving data...
Loading centroids...
Mapping neurons in brain regions...
100%|███████████████████████████████████████████| 70/70 [00:27<00:00,  2.55it/s]
Saving data...
Loading centroids...
Mapping neurons in brain regions...
100%|████████████████████